In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,513170
1,Huelva,Confirmados PDIA,71011
2,Huelva,Confirmados PDIA 14 días,7677
3,Huelva,Tasa PDIA 14 días,"1495,9954790810064"
4,Huelva,Confirmados PDIA 7 días,3180
5,Huelva,Tasa PDIA 7 dias,"619,6776896545005"
6,Huelva,Total Confirmados,71245
7,Huelva,Curados,54518
8,Huelva,Fallecidos,437


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  71011.0


/tmp/ipykernel_29920/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  20773.0


/tmp/ipykernel_29920/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_29920/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_29920/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_29920/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 148 personas en los últimos 7 días 

Un positivo PCR cada 60 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,71011.0,7677.0,3180.0,513170.0,619.677690,1495.995479,695.0
Huelva-Costa,41223.0,4058.0,1686.0,289548.0,582.286875,1401.494744,334.0
Sierra de Huelva-Andévalo Central,9024.0,1947.0,865.0,67391.0,1283.554184,2889.109822,234.0
Huelva (capital),20773.0,2387.0,971.0,143837.0,675.069697,1659.517370,203.0
Condado-Campiña,20299.0,1663.0,625.0,156231.0,400.048646,1064.449437,128.0
Valverde del Camino,1970.0,375.0,166.0,12750.0,1301.960784,2941.176471,55.0
Aracena,1326.0,190.0,96.0,8255.0,1162.931557,2301.635373,25.0
Almonte,3106.0,247.0,95.0,24507.0,387.644347,1007.875301,23.0
Isla Cristina,3871.0,220.0,101.0,21393.0,472.117048,1028.373767,22.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cerro de Andévalo (El),442.0,134.0,54.0,2327.0,2320.584443,5758.487323,11.0
Cortegana,672.0,194.0,79.0,4602.0,1716.644937,4215.558453,18.0
Jabugo,280.0,94.0,39.0,2260.0,1725.663717,4159.292035,11.0
Nerva,636.0,205.0,84.0,5169.0,1625.072548,3965.950861,22.0
Rosal de la Frontera,236.0,66.0,29.0,1697.0,1708.898055,3889.216264,6.0
Marines (Los),39.0,14.0,8.0,399.0,2005.012531,3508.771930,3.0
Valdelarco,13.0,8.0,7.0,231.0,3030.303030,3463.203463,2.0
Cumbres Mayores,294.0,59.0,11.0,1749.0,628.930818,3373.356204,3.0
Aroche,351.0,99.0,47.0,3054.0,1538.965291,3241.650295,17.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Paymogo,120.0,15.0,2.0,1162.0,172.117040,1290.877797,0.0,0.133333
Santa Bárbara de Casa,69.0,7.0,1.0,1043.0,95.877277,671.140940,0.0,0.142857
Cumbres Mayores,294.0,59.0,11.0,1749.0,628.930818,3373.356204,3.0,0.186441
Higuera de la Sierra,121.0,9.0,2.0,1291.0,154.918668,697.134005,1.0,0.222222
San Bartolomé de la Torre,504.0,29.0,7.0,3761.0,186.120713,771.071524,2.0,0.241379
Gibraleón,1484.0,190.0,46.0,12737.0,361.152548,1491.717045,7.0,0.242105
Nava (La),15.0,4.0,1.0,258.0,387.596899,1550.387597,0.0,0.250000
Beas,526.0,52.0,13.0,4341.0,299.470168,1197.880673,2.0,0.250000
Fuenteheridos,81.0,22.0,6.0,709.0,846.262341,3102.961918,1.0,0.272727
